In [13]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

# データの読み込み
modelling_table = pd.read_csv("preprocessed_data.csv")

# 太陽光発電と風力発電のための別々のモデル
solar_mod = smf.quantreg('Solar_MWh_credit ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5)', data=modelling_table)
wind_mod = smf.quantreg('Wind_MWh_credit ~ bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=modelling_table)

forecast_models_solar = dict()
forecast_models_wind = dict()

# 太陽光エネルギーと風力エネルギーの分位数を別々に計算
for quantile in range(10, 100, 10):
    forecast_models_solar[f"q{quantile}"] = solar_mod.fit(q=quantile/100, max_iter=2500)
    forecast_models_wind[f"q{quantile}"] = wind_mod.fit(q=quantile/100, max_iter=2500)

    modelling_table[f"q{quantile}_solar"] = forecast_models_solar[f"q{quantile}"].predict(modelling_table)
    modelling_table[f"q{quantile}_wind"] = forecast_models_wind[f"q{quantile}"].predict(modelling_table)

    # 発電量は0以上
    modelling_table.loc[modelling_table[f"q{quantile}_solar"] < 0, f"q{quantile}_solar"] = 0
    modelling_table.loc[modelling_table[f"q{quantile}_wind"] < 0, f"q{quantile}_wind"] = 0

    # 小数点3桁で丸める
    modelling_table[f"q{quantile}_solar"] = modelling_table[f"q{quantile}_solar"].round(2)
    modelling_table[f"q{quantile}_wind"] = modelling_table[f"q{quantile}_wind"].round(2)

    # 太陽光と風力の発電量の合計をトータル発電量として計算
    modelling_table[f"q{quantile}"] = modelling_table[f"q{quantile}_solar"] + modelling_table[f"q{quantile}_wind"]

# 太陽光エネルギーと風力エネルギーの予測を分けて保存した更新データを保存
modelling_table.to_csv("analyzed_data_sep.csv", index=False)

# 太陽光エネルギーと風力エネルギーのための分位数回帰モデルをファイルに保存
for quantile in range(10,100,10):
    forecast_models_solar[f"q{quantile}"].save(f"models_sep/solar_model_q{quantile}.pickle")
    forecast_models_wind[f"q{quantile}"].save(f"models_sep/wind_model_q{quantile}.pickle")

/usr/local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (5000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
